In [2]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
from dataset_working import ImageNetDataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import PIL
import os
from pathlib import Path
from typing import Callable, List, Tuple, Dict
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import torch

import wandb
from pytorch_metric_learning import distances, losses, miners, reducers
import pandas as pd


import matplotlib.pyplot as plt

from torch import nn
import torch.optim as optim
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from turtle import color
import numpy as np
import tqdm as tqdm
import copy
from sklearn.metrics import f1_score
import shutil
from inspect import stack
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle

from torch.utils.data import Dataset, DataLoader
import PIL
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.model_selection import train_test_split

/home/vlad/GarsegVenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
np.random.seed(42)
torch.seed()

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
torch.cuda.set_device(1)

In [4]:
data_dir = 'dataset'
data_list = 'dataset_list.csv'
classes = {'trucks':0, 'minibus':1, 'ski':2, 'dump_trucks':3, 'bicycles':4, 'snowboard':5, 'tractor':6, 'trains':7, 'gazon':8, 'horses':9}
classes_inv = {v: k for k, v in classes.items()}


normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
input_transform = transforms.Compose([
    transforms.Resize(256, PIL.Image.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])


dataset_list = pd.read_csv(data_list)
train_data, val_test_data = train_test_split(dataset_list, test_size = 0.2, shuffle = True)
val_data, test_data = train_test_split(val_test_data, test_size = 0.5, shuffle = True)

dataset_train = ImageNetDataset(data_dir, train_data, classes, transform = input_transform)
dataset_val = ImageNetDataset(data_dir,val_data, classes, transform = input_transform)
dataset_test= ImageNetDataset(data_dir, test_data, classes, transform = input_transform)


image_datasets = {'train':dataset_train, 'val':dataset_val} #тут надо вставить датасеты

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = classes_inv #это может не работать

/tmp/ipykernel_3734153/2795392040.py:11: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  transforms.Resize(256, PIL.Image.BICUBIC),
/home/vlad/GarsegVenv/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [5]:
N_CLASSES = 10

model_ft = torchvision.models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, N_CLASSES)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/home/vlad/GarsegVenv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/vlad/GarsegVenv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
from train import train_model

model_ft = train_model(model_ft, device, dataloaders, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6739 Acc: 0.8078
val Loss: 0.2800 Acc: 0.9245

Epoch 1/24
----------
train Loss: 0.2252 Acc: 0.9373
val Loss: 0.1958 Acc: 0.9470

Epoch 2/24
----------
train Loss: 0.1500 Acc: 0.9598
val Loss: 0.1589 Acc: 0.9579

Epoch 3/24
----------
train Loss: 0.1039 Acc: 0.9740
val Loss: 0.1380 Acc: 0.9634

Epoch 4/24
----------
train Loss: 0.0733 Acc: 0.9850
val Loss: 0.1308 Acc: 0.9595

Epoch 5/24
----------
train Loss: 0.0531 Acc: 0.9906
val Loss: 0.1224 Acc: 0.9642

Epoch 6/24
----------
train Loss: 0.0403 Acc: 0.9943
val Loss: 0.1196 Acc: 0.9650

Epoch 7/24
----------
train Loss: 0.0294 Acc: 0.9976
val Loss: 0.1153 Acc: 0.9657

Epoch 8/24
----------
train Loss: 0.0298 Acc: 0.9975
val Loss: 0.1175 Acc: 0.9650

Epoch 9/24
----------
train Loss: 0.0283 Acc: 0.9975
val Loss: 0.1172 Acc: 0.9665

Epoch 10/24
----------
train Loss: 0.0278 Acc: 0.9979
val Loss: 0.1174 Acc: 0.9650

Epoch 11/24
----------
train Loss: 0.0272 Acc: 0.9977
val Loss: 0.1133 Acc: 0.9673

Ep

In [ ]:
from visualize import visualize_model

visualize_model(model_ft, device, dataloaders, class_names)